In [4]:
import sys
sys.path.append(r"C:\Users\Aditya Gautam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages")

import yfinance as yf
print("Yes")
import sys
print(sys.executable)


Yes
c:\ProgramData\anaconda3\python.exe


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from ta.trend import ADXIndicator, SMAIndicator
from ta.momentum import StochasticOscillator
from ta.volatility import BollingerBands
from ta.utils import dropna

# Load and preprocess data
def load_data(ticker, start_date, end_date):
    """Load and preprocess OHLCV data using yfinance."""
    data = yf.download(ticker, start=start_date, end=end_date)
    data.rename(columns={'Adj Close': 'close'}, inplace=True)
    data = dropna(data)
    return data

# Add Technical Indicators
def add_indicators(data):
    """Add ADX, Bollinger Bands, Stochastic Oscillator, and Supertrend."""
    # ADX
    adx = ADXIndicator(high=data['High'], low=data['Low'], close=data['close'], window=14)
    data['adx'] = adx.adx()
    data['adx_pos'] = adx.adx_pos()
    data['adx_neg'] = adx.adx_neg()

    # Bollinger Bands
    bb = BollingerBands(close=data['close'], window=20, window_dev=2)
    data['bb_upper'] = bb.bollinger_hband()
    data['bb_lower'] = bb.bollinger_lband()
    data['bb_mid'] = bb.bollinger_mavg()

    # Stochastic Oscillator
    stoch = StochasticOscillator(high=data['High'], low=data['Low'], close=data['close'], window=14, smooth_window=3)
    data['stoch_k'] = stoch.stoch()
    data['stoch_d'] = stoch.stoch_signal()

    # Supertrend
    atr = SMAIndicator(close=data['close'], window=10).sma_indicator()
    multiplier = 3
    data['supertrend'] = np.where(
        (data['close'] > data['High'] - multiplier * atr), 1, -1
    )

    return data

# Plot Heatmap
def plot_heatmap(data):
    """Generate a correlation heatmap for OHLCV data with technical indicators."""
    plt.figure(figsize=(12, 8))
    sns.heatmap(data.corr(), annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Correlation Heatmap")
    plt.show()

# Strategy Logic
def strategy(data):
    """Trading strategy based on indicators."""
    buy_signal = (data['stoch_k'] < 20) & (data['adx'] > 25) & (data['close'] < data['bb_lower'])
    sell_signal = (data['stoch_k'] > 80) & (data['adx'] > 25) & (data['close'] > data['bb_upper'])
    
    data['signal'] = 0
    data.loc[buy_signal, 'signal'] = 1
    data.loc[sell_signal, 'signal'] = -1
    return data

# Backtest Metrics
def backtest(data):
    """Calculate backtest metrics."""
    data['daily_return'] = data['close'].pct_change()
    data['strategy_return'] = data['signal'].shift(1) * data['daily_return']

    cumulative_strategy_return = (1 + data['strategy_return']).cumprod()
    cumulative_buy_and_hold = (1 + data['daily_return']).cumprod()

    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_strategy_return, label="Strategy Return")
    plt.plot(cumulative_buy_and_hold, label="Buy and Hold Return")
    plt.legend()
    plt.title("Strategy vs Buy and Hold Performance")
    plt.show()

    sharpe_ratio = data['strategy_return'].mean() / data['strategy_return'].std() * np.sqrt(252)
    print(f"Sharpe Ratio: {sharpe_ratio}")

# Preprocess for LSTM
def preprocess_data(data, feature='close', sequence_length=60):
    """Preprocess data for LSTM model."""
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[[feature]].values)
    
    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])
    return np.array(X), np.array(y), scaler

# LSTM Model
def create_lstm_model(input_shape):
    """Create and compile LSTM model."""
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Output layer
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Main Execution
if __name__ == "__main__":
    # Load and preprocess data
    ticker = "BTC-USD"  # Replace with your ticker symbol
    start_date = "2020-01-01"
    end_date = "2023-01-01"
    data = load_data(ticker, start_date, end_date)
    data = add_indicators(data)

    # Plot Heatmap
    plot_heatmap(data)

    # Apply Strategy
    data = strategy(data)

    # Backtest Strategy
    backtest(data)

    # Preprocess for LSTM
    feature = 'close'
    sequence_length = 60
    X, y, scaler = preprocess_data(data, feature, sequence_length)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Train-Test Split
    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Create and Train LSTM Model
    model = create_lstm_model((X_train.shape[1], 1))
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

    # Predictions
    predictions = model.predict(X_test)
    predictions_inverse = scaler.inverse_transform(predictions)
    y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Evaluate Model
    mse = mean_squared_error(y_test_inverse, predictions_inverse)
    mae = mean_absolute_error(y_test_inverse, predictions_inverse)
    print(f"Model MSE: {mse}")
    print(f"Model MAE: {mae}")

    # Plot Predictions
    plt.figure(figsize=(12, 6))
    plt.plot(y_test_inverse, label="Actual Prices")
    plt.plot(predictions_inverse, label="Predicted Prices", alpha=0.7)
    plt.title("LSTM Price Prediction vs Actual")
    plt.legend()
    plt.show()


ModuleNotFoundError: No module named 'tensorflow'